In [ ]:
Import libraries

In [ ]:
import torch
from ultralytics import YOLO
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt
import cv2
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

In [ ]:
Load YOLO model

In [ ]:
yolo_model = YOLO("yolov8n.pt")

In [ ]:
Load Stable Diffusion model

In [ ]:
model_id = "runwayml/stable-diffusion-v1-5"

sd_pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)
sd_pipe = sd_pipe.to(device)

In [ ]:
Synthetic image generator

In [ ]:
def generate_synthetic_room(prompt, filename, steps=30):
    image = sd_pipe(prompt, num_inference_steps=steps, guidance_scale=7.5).images[0]
    image.save(filename)
    print(f"Saved image: {filename}")
    return filename

In [ ]:
EXAMPLE PROMPT:

In [ ]:
prompts = [
    "a child's room with toys, scissors on table, realistic",
    "nursery with toys and a small battery on the carpet",
    "kids room with open cleaning liquid bottle near toys"
]

files = []
for i, p in enumerate(prompts, start=1):
    files.append(generate_synthetic_room(p, f"synthetic_{i}.png"))

In [ ]:
YOLO inference

In [ ]:
def show_yolo_result(image_path, model, conf=0.25):
    results = model.predict(image_path, conf=conf)[0]
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        cls_id = int(box.cls[0].cpu().numpy())
        conf_score = float(box.conf[0].cpu().numpy())
        label = model.names[cls_id]

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img, f"{label} {conf_score:.2f}",
                    (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0, 255, 0), 1)

    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.axis("off")
    plt.show()

In [ ]:
Run inference:

In [ ]:
for f in files:
    show_yolo_result(f, yolo_model, conf=0.3)